In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score,confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score,precision_recall_curve
from sklearn.ensemble import RandomForestClassifier , StackingClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression,Lasso
from sklearn.preprocessing import Binarizer
from sklearn.model_selection import cross_val_score,GridSearchCV

# 한글 깨짐 방지
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'malgun Gothic'

In [2]:
train=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_train.csv',index_col=False, encoding='euc-kr')
test=pd.read_csv('/Users/taewon/Documents/금융 빅데이터/Project_2/코딩/상장기업/sampling data/Undersampling_0.33_test.csv',index_col=False,encoding='euc-kr')

In [3]:
X_train_int=train[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

X_test_int=test[['부가가치율', '자기자본구성비율', '총자산대비영업현금흐름', '자기자본순이익률', '총자본순이익률', '총자본회전률',
       'PBR', '이윤분배율', '자기자본증가율', '차입금의존도', 'log자산총계', 'PER', '연구개발비대비매출액',
       '유동자산회전률']]

In [4]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [5]:
X_train_sc = X_train
X_test_sc = X_test

In [6]:
X_train_sum = X_train_sc
X_test_sum = X_test_sc

In [7]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate_tabnet(X_train, y_train, X_test, y_test):
    # Stratified 5-fold 교차검증 설정
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

    # 최적 하이퍼파라미터 설정 (여기서는 고정값으로 사용)
    best_params = {
        'n_d': 8,  # Number of decision steps (also known as the number of features for attention)
        'n_a': 8,  # Number of attention features (output dimension of each attention head)
        'n_steps': 3,  # Number of steps in the architecture (usually between 3 and 10)
        'gamma': 1.5,  # The factor by which to scale the contribution of each augmented sample
        'n_independent': 2,  # Number of independent GLU layers in each decision step
        'n_shared': 2,  # Number of shared GLU layers in each decision step
        'lambda_sparse': 0.001  # The sparsity loss weight
    }

    # 각 fold 별 평가 지표를 저장할 리스트 초기화
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # TabNet 모델 초기화
        model = TabNetClassifier(
            n_d=best_params['n_d'],
            n_a=best_params['n_a'],
            n_steps=best_params['n_steps'],
            gamma=best_params['gamma'],
            n_independent=best_params['n_independent'],
            n_shared=best_params['n_shared'],
            lambda_sparse=best_params['lambda_sparse'],
            verbose=0
        )

        # 모델 학습
        model.fit(
            X_train_fold.values, y_train_fold.values.ravel(),  # Convert y_train to 1D array using .ravel()
            eval_set=[(X_test_fold.values, y_test_fold.values.ravel())],  # Convert y_test to 1D array using .ravel()
            eval_metric=['accuracy'],
            patience=100,
            batch_size=1024,
            virtual_batch_size=128,
            max_epochs=1000
        )

        # 테스트 데이터에 대한 예측
        y_pred = model.predict(X_test_fold.values)

        # 평가 지표 계산
        accuracy = accuracy_score(y_test_fold, y_pred)
        precision = precision_score(y_test_fold, y_pred)
        recall = recall_score(y_test_fold, y_pred)
        f1 = f1_score(y_test_fold, y_pred)
        conf_matrix = confusion_matrix(y_test_fold, y_pred)

        # 각 fold 별 평가 지표를 리스트에 추가
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # 가장 좋은 f1-score 값을 가진 모델을 저장
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # 가장 좋은 f1-score 값을 가진 모델로 최종 예측 수행
    y_pred_final = best_model.predict(X_test.values)

    # 평가 지표 계산
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)  # 테스트 데이터에 대한 Confusion Matrix 계산

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list

In [8]:
# 함수를 호출하여 cross validation과 test를 수행합니다.
evaluate_tabnet(X_train_sum, y_train, X_test_sum, y_test)


Early stopping occurred at epoch 100 with best_epoch = 0 and best_val_0_accuracy = 0.37838
Fold 1
Accuracy: 0.3783783783783784
Precision: 0.26666666666666666
Recall: 0.8888888888888888
F1 score: 0.41025641025641024
Confusion Matrix:
[[ 6 22]
 [ 1  8]]
------------------------------

Early stopping occurred at epoch 100 with best_epoch = 0 and best_val_0_accuracy = 0.27027
Fold 2
Accuracy: 0.2702702702702703
Precision: 0.15384615384615385
Recall: 0.4444444444444444
F1 score: 0.2285714285714286
Confusion Matrix:
[[ 6 22]
 [ 5  4]]
------------------------------

Early stopping occurred at epoch 100 with best_epoch = 0 and best_val_0_accuracy = 0.41667
Fold 3
Accuracy: 0.4166666666666667
Precision: 0.2692307692307692
Recall: 0.7777777777777778
F1 score: 0.39999999999999997
Confusion Matrix:
[[ 8 19]
 [ 2  7]]
------------------------------

Early stopping occurred at epoch 100 with best_epoch = 0 and best_val_0_accuracy = 0.44444
Fold 4
Accuracy: 0.4444444444444444
Precision: 0.260869565

([0.3783783783783784,
  0.2702702702702703,
  0.4166666666666667,
  0.4444444444444444,
  0.3333333333333333,
  0.3888888888888889,
  0.4722222222222222,
  0.3055555555555556,
  0.2777777777777778,
  0.4444444444444444],
 [0.26666666666666666,
  0.15384615384615385,
  0.2692307692307692,
  0.2608695652173913,
  0.2,
  0.19047619047619047,
  0.2916666666666667,
  0.21428571428571427,
  0.13043478260869565,
  0.2962962962962963],
 [0.8888888888888888,
  0.4444444444444444,
  0.7777777777777778,
  0.6666666666666666,
  0.5555555555555556,
  0.4444444444444444,
  0.7777777777777778,
  0.6666666666666666,
  0.3333333333333333,
  0.8888888888888888],
 [0.41025641025641024,
  0.2285714285714286,
  0.39999999999999997,
  0.37500000000000006,
  0.29411764705882354,
  0.26666666666666666,
  0.42424242424242425,
  0.3243243243243243,
  0.18750000000000003,
  0.4444444444444444])